# Base classes
> Base configuration classes and utilities for the logging library.

In [ ]:
#| default_exp client.base

In [ ]:
#| exporti

from typing import Optional, List, Dict, Any, Literal
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from enum import Enum


In [ ]:
#| export

@dataclass
class LoggerSettings(ABC):
    """abstract base configuration for logging configuration settings"""

    format: str = "json"  # json, text
    batch_size: int = 100
    flush_interval: int = 30  # seconds
    correlation_enabled: bool = True
    include_traceback: bool = True
    max_buffer_size: int = 1000

In [ ]:
#| export


# Type for valid output modes
OutputMode = Literal["cloud", "console", "file", "multi"]


@dataclass
class ServiceConfig(ABC):
    """abstract base class for service-specific configuration settings"""

    output_mode: OutputMode

    def __post_init__(self):
        self.validate_config()


    @abstractmethod
    def validate_config(self) -> bool:
        """Validate the configuration"""
        raise NotImplementedError()

In [ ]:
#| export

class LogLevel(str, Enum):
    """Standard logging levels"""

    DEBUG = "DEBUG"
    INFO = "INFO"
    WARNING = "WARNING"
    ERROR = "ERROR"
    CRITICAL = "CRITICAL"

    @classmethod
    def from_string(cls, level_str: str) -> "LogLevel":
        """Convert string to LogLevel enum"""
        try:
            return cls(level_str.upper())
        except ValueError:
            return cls.INFO  # default fallback

    def should_log(self, other: "LogLevel") -> bool:
        """Check if this level should log the other level"""
        levels = list(LogLevel)
        return levels.index(self) <= levels.index(other)

In [ ]:
#| export

    
@dataclass
class LogHandler:
    """defines when, how and where to send log entries"""

    log_config: LoggerSettings
    service_config: ServiceConfig

    log_level: LogLevel = LogLevel.INFO


@dataclass 
class Logger:
    handlers: List[LogHandler] = field(default_factory=list)
    pretty_print: bool = False  # Pretty print JSON for development

In [4]:
#| hide
import nbdev; nbdev.nbdev_export('./base.ipynb')